In [ ]:
from pathlib import Path
BASEPATH = Path("/Users/joregan/Playing/irish-gists/scrape-fuaimeanna-private/")
OUTPATH = Path("/tmp/fuaimeanna-fairseq")

In [14]:
if not OUTPATH.is_dir():
    OUTPATH.mkdir()

In [9]:
EMPTY_AUDIO = """
gob_i3_s3.mp3
iioctha_i3_s3.mp3
mo_shuiiochaan_i3_s3.mp3
riail_i3_s3.mp3
""".strip().split("\n")

In [7]:
PHONES_NO_AUDIO = """
d'fhaag_i1_s1.phones
d'fhaag_i2_s2.phones
d'fhaag_i3_s3.phones
""".strip().split("\n")

In [3]:
parameters=["-ac", "1", "-acodec", "pcm_s16le", "-ar", "16000"]

In [5]:
from pydub import AudioSegment

In [25]:
def drop_accents(pron):
    parts = pron.split(" ")
    return " ".join([x for x in parts if x not in [".", "ˈ"]])

In [26]:
prons_gd = {}
prons_cr = {}
prons_cd = {}
with open(str(BASEPATH / "all-fuaimeanna-data.tsv")) as tsvf:
    for line in tsvf.readlines():
        if line.startswith("Orth"):
            continue
        parts = line.split("\t")
        fname = parts[1]
        fname = fname.replace(".mp3", "").split("/")[2]
        prons_gd[fname] = drop_accents(parts[2].strip())
        fname = parts[3]
        fname = fname.replace(".mp3", "").split("/")[2]
        prons_cr[fname] = drop_accents(parts[4].strip())
        fname = parts[5]
        fname = fname.replace(".mp3", "").split("/")[2]
        prons_cd[fname] = drop_accents(parts[6].strip())


In [29]:
prons_all = {**prons_cd, **prons_cr, **prons_gd}

In [53]:
# get a validation set from 5% of each
validkeys = list(prons_cd.keys())[:int(len(prons_cd) * .05)]
validkeys += list(prons_cr.keys())[:int(len(prons_cr) * .05)]
validkeys += list(prons_gd.keys())[:int(len(prons_gd) * .05)]

In [31]:
alldata = []
for audio_file in (BASEPATH / "mp3").glob("*.mp3"):
    if audio_file.name in EMPTY_AUDIO:
        continue
    stem = audio_file.stem
    if f"{stem}.phones" in PHONES_NO_AUDIO:
        continue
    data = {}
    data["name"] = stem
    spoken = AudioSegment.from_mp3(str(audio_file))
    outfile = OUTPATH / f"{stem}.wav"
    spoken.export(str(outfile), format="wav", parameters=parameters)
    data["labels"] = prons_all[stem].replace("#", "|")
    data["frames"] = spoken.frame_count()
    alldata.append(data)